In [ ]:
"""
Grazioso Salvare Animal Rescue Dashboard

This interactive dashboard enables Grazioso Salvare to identify and categorize
rescue dog candidates from Austin Animal Center shelter data. The dashboard
filters animals by rescue specialization (water, mountain/wilderness, disaster
rescue, or individual tracking) based on breed, sex, age, and intact status.

Architecture:
    Model-View-Controller (MVC) pattern implemented in single Jupyter cell
    - Model: MongoDB data access via AnimalShelter CRUD module
    - View: Dash/Plotly interactive components (login, filters, table, charts)
    - Controller: Dash callbacks coordinating user interactions

Features:
    - Authentication gate with username/password validation
    - Four rescue type filters with specific breed/sex/age criteria
    - Interactive data table with sorting, pagination, row selection
    - Geolocation map showing selected animal's location
    - Pie chart displaying outcome type distribution for filtered data
    - Row highlighting for selected animal
    - Responsive empty states when no data/selection

Authentication:
    Username: admin
    Password: grazioso2024

Rescue Filter Criteria:
    Water Rescue:
        Breeds: Labrador Retriever, Chesapeake Bay Retriever, Newfoundland
        Sex: Intact Female
        Age: 26-156 weeks

    Mountain/Wilderness Rescue:
        Breeds: German Shepherd, Alaskan Malamute, Old English Sheepdog,
                Siberian Husky, Rottweiler
        Sex: Intact Male
        Age: 26-156 weeks

    Disaster Rescue / Individual Tracking:
        Breeds: Doberman Pinscher, German Shepherd, Golden Retriever,
                Bloodhound, Rottweiler
        Sex: Intact Male
        Age: 20-300 weeks

    Reset: Shows all animals (no filtering)

Data Flow:
    1. Raw data fetched from MongoDB (aac.animals collection)
    2. Data normalized (age_weeks, sex, intact_status, valid_coords)
    3. User selects rescue filter via radio buttons
    4. apply_rescue_filter() applies breed/sex/age criteria
    5. Filtered data updates table, chart, and map
    6. User selects table row to view animal location on map

Author: Rick Goshen
Course: CS 340 - Client/Server Development
Institution: Southern New Hampshire University
"""

###########################
# Imports and Setup
###########################
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import base64

# Configure data manipulation
import numpy as np
import pandas as pd

# Import CRUD module
from CRUD_Python_Module import AnimalShelter

# Import helper modules
from data_helpers import normalize_dataframe, bucket_categories
from rescue_filters import apply_rescue_filter
from dashboard_auth import validate_credentials, get_auth_error_message, is_authenticated


###########################
# Data Manipulation / Model
###########################
# Database credentials (coursework - not for production)
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Read all documents from database
df_raw = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB _id column (ObjectID type causes issues with dash_table)
df_raw.drop(columns=['_id'], inplace=True)

# Normalize the dataframe (create age_weeks, sex, intact_status, valid_coords columns)
df = normalize_dataframe(df_raw)


###########################
# Dashboard Layout / View
###########################
app = JupyterDash(__name__, suppress_callback_exceptions=True)

# Add Leaflet CSS for map rendering
app.css.append_css({
    'external_url': 'https://unpkg.com/leaflet@1.9.4/dist/leaflet.css'
})

# Load and encode logo for display
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Main Dashboard Layout - contains all visual components after authentication
dashboard_layout = html.Div(id='dashboard-content', children=[
    # Header with logo and branding
    html.Div([
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo',
                     style={'height': '80px', 'display': 'block', 'margin': '0 auto'})
        ], href='https://www.snhu.edu', target='_blank'),
        html.H1('Grazioso Salvare Animal Rescue Dashboard',
                style={'textAlign': 'center', 'color': '#2c3e50', 'marginTop': '20px'}),
        html.P('Dashboard by Rick Goshen',
               style={'textAlign': 'center', 'fontStyle': 'italic', 'color': '#7f8c8d'}),
        html.P('CS 340 - Client/Server Development',
               style={'textAlign': 'center', 'color': '#95a5a6', 'fontSize': '14px'}),
    ], style={'padding': '20px', 'backgroundColor': '#ecf0f1', 'borderRadius': '10px', 'marginBottom': '20px'}),

    html.Hr(),

    # Filter Controls - radio buttons for rescue type selection
    html.Div([
        html.H3('Select Rescue Type Filter:', style={'color': '#34495e'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Water Rescue (Labrador, Chesapeake Bay Retriever, Newfoundland - Intact Female, 26-156 weeks)',
                 'value': 'water'},
                {'label': ' Mountain/Wilderness Rescue (German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler - Intact Male, 26-156 weeks)',
                 'value': 'mountain'},
                {'label': ' Disaster Rescue or Individual Tracking (Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler - Intact Male, 20-300 weeks)',
                 'value': 'disaster'},
                {'label': ' Reset (Show All Animals)', 'value': 'reset'}
            ],
            value='reset',  # Default to showing all animals
            labelStyle={'display': 'block', 'marginBottom': '10px'},
            style={'padding': '15px'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px', 'marginBottom': '20px'}),

    html.Hr(),

    # Data Table - interactive table with sorting, pagination, selection
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False,
                  "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',  # Only one row can be selected at a time
        selected_rows=[0],  # First row selected by default
        sort_action='native',  # Enable column sorting
        page_action='native',  # Enable pagination
        page_current=0,
        page_size=10,  # Show 10 rows per page
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px',
            'width': '150px',
            'maxWidth': '200px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        # Tooltips show full content on hover for truncated cells
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None  # Tooltips stay visible while hovering
    ),

    html.Br(),
    html.Hr(),

    # Charts side-by-side - pie chart and geolocation map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),  # Outcome type pie chart
        html.Div(id='map-id', className='col s12 m6')     # Geolocation map
    ])
])

# Main App Layout - both login and dashboard always present in DOM, visibility toggled
app.layout = html.Div([
    # Store authentication state (not visible to user)
    dcc.Store(id='auth-state', data={'authenticated': False}),
    # Store previous page number to detect actual page navigation
    dcc.Store(id='previous-page', data=0),
    
    # Login Container (always in DOM, visibility toggled via CSS)
    html.Div(id='login-container', children=[
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     alt='Grazioso Salvare Logo',
                     style={'height': '100px', 'display': 'block', 'margin': '20px auto'}),
            html.H2('Grazioso Salvare Dashboard Login',
                    style={'textAlign': 'center', 'color': '#2c3e50'}),
            html.Hr(),
            html.Div([
                html.Label('Username:', style={
                           'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Input(id='username-input', type='text', placeholder='Enter username',
                          style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
                html.Label('Password:', style={
                           'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Input(id='password-input', type='password', placeholder='Enter password',
                          style={'width': '100%', 'padding': '10px', 'marginBottom': '15px'}),
                html.Button('Login', id='login-button', n_clicks=0,
                            style={'width': '100%', 'padding': '10px', 'backgroundColor': '#3498db',
                                   'color': 'white', 'border': 'none', 'borderRadius': '5px',
                                   'fontSize': '16px', 'cursor': 'pointer'}),
                html.Div(id='login-error',
                        style={'color': 'red', 'marginTop': '15px', 'textAlign': 'center'})
            ], style={'maxWidth': '400px', 'margin': '0 auto', 'padding': '30px',
                      'backgroundColor': '#ecf0f1', 'borderRadius': '10px'})
        ], style={'padding': '50px'})
    ], style={'display': 'block'}),
    
    # Dashboard Container (always in DOM, visibility toggled via CSS)
    html.Div(id='dashboard-container', children=dashboard_layout, style={'display': 'none'})
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('auth-state', 'data'),
     Output('login-error', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('username-input', 'value'),
     State('password-input', 'value')]
)
def authenticate_user(n_clicks, username, password):
    """
    Handle user authentication and update auth state.
    
    Validates credentials against coursework authentication (admin/grazioso2024).
    Returns authentication state and error message to display to user.
    
    Args:
        n_clicks: Number of times login button has been clicked
        username: Username entered by user
        password: Password entered by user
        
    Returns:
        tuple: (auth_state dict, error_message string)
            auth_state: {'authenticated': True/False}
            error_message: Empty string on success, error message on failure
    """
    # Don't authenticate on initial page load (n_clicks=0)
    if n_clicks == 0:
        return {'authenticated': False}, ''

    # Validate credentials using dashboard_auth module
    if validate_credentials(username, password):
        return {'authenticated': True}, ''
    else:
        error_msg = get_auth_error_message(username, password)
        return {'authenticated': False}, error_msg


@app.callback(
    [Output('login-container', 'style'),
     Output('dashboard-container', 'style')],
    [Input('auth-state', 'data')]
)
def toggle_screens(auth_state):
    """
    Toggle visibility between login screen and dashboard based on authentication.
    
    Uses CSS display property to show/hide containers without destroying DOM.
    This preserves input values across authentication attempts.
    
    Args:
        auth_state: Dictionary with 'authenticated' boolean flag
        
    Returns:
        tuple: (login_style dict, dashboard_style dict)
            Each contains 'display' key set to 'block' or 'none'
    """
    if is_authenticated(auth_state):
        # Hide login, show dashboard
        return {'display': 'none'}, {'display': 'block'}
    else:
        # Show login, hide dashboard
        return {'display': 'block'}, {'display': 'none'}


@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'page_current'),
     Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Apply rescue type filter and reset table state.
    
    Filters the normalized dataframe based on selected rescue type using
    apply_rescue_filter() dispatcher. Resets pagination to first page and
    selects first row to ensure consistent UX after filter changes.
    
    Args:
        filter_type: String indicating rescue type
            'water', 'mountain', 'disaster', or 'reset'
            
    Returns:
        tuple: (filtered_data, page_number, selected_rows)
            filtered_data: List of dicts for DataTable
            page_number: Always 0 (first page)
            selected_rows: Always [0] (first row selected)
    """
    try:
        # Apply the appropriate filter using rescue_filters module
        filtered_df = apply_rescue_filter(df, filter_type)
        # Return filtered data, reset to page 0, and select first row
        return filtered_df.to_dict('records'), 0, [0]
    except Exception:
        # On error, return full dataset, reset to page 0, select first row
        return df.to_dict('records'), 0, [0]


@app.callback(
    [Output('datatable-id', 'selected_rows', allow_duplicate=True),
     Output('previous-page', 'data')],
    [Input('datatable-id', 'page_current')],
    [State('previous-page', 'data'),
     State('datatable-id', 'selected_rows')],
    prevent_initial_call=True
)
def clear_selection_on_page_change(current_page, previous_page, current_selection):
    """
    Clear row selection when user navigates to different page.
    
    Prevents highlighting from persisting on rows with same index across pages.
    Only clears on actual page navigation, not on filter changes or initial load.
    
    Args:
        current_page: Current page number (0-indexed)
        previous_page: Previous page number stored in dcc.Store
        current_selection: Current selected_rows list
        
    Returns:
        tuple: (selected_rows, previous_page)
            selected_rows: Empty list [] if page changed, otherwise unchanged
            previous_page: Updated to current_page value
    """
    # Only clear if this is an actual page change (not initial load or filter change)
    if previous_page != current_page:
        # Clear selection and update previous page tracker
        return [], current_page
    else:
        # No change, keep current selection
        return current_selection, current_page


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """
    Update outcome type distribution pie chart based on filtered data.
    
    Creates pie chart showing distribution of outcome types in currently
    filtered/sorted data. Uses category bucketing to limit chart to top 10
    categories plus "Other" for readability.
    
    Args:
        viewData: List of dicts representing currently visible table data
            (after filtering, sorting, pagination)
            
    Returns:
        dcc.Graph or html.Div: Pie chart component or empty state message
    """
    try:
        # Use full dataset as fallback if derived_virtual_data is None
        if viewData is None:
            viewData = df.to_dict('records')
        
        # Show empty state if no data to display
        if len(viewData) == 0:
            return html.Div([
                html.H4('No data to display', style={
                        'textAlign': 'center', 'color': '#95a5a6'})
            ])

        dff = pd.DataFrame.from_dict(viewData)

        # Get outcome type values as list for bucket_categories function
        outcome_values = dff['outcome_type'].tolist()

        # Apply category bucketing (top 10 + Other) using data_helpers module
        category_mapping = bucket_categories(outcome_values, top_n=10)
        dff['outcome_bucketed'] = dff['outcome_type'].map(category_mapping)

        # Create pie chart with Plotly Express
        fig = px.pie(
            dff,
            names='outcome_bucketed',
            title='Outcome Type Distribution',
            color_discrete_sequence=px.colors.qualitative.Set3
        )

        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.update_layout(showlegend=True, height=500)

        return dcc.Graph(figure=fig)
    except Exception:
        # Show error state if chart rendering fails
        return html.Div([
            html.H4('Error loading chart', style={
                    'textAlign': 'center', 'color': '#e74c3c'})
        ])


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_virtual_selected_rows')]
)
def highlight_selected_row(derived_virtual_selected_rows):
    """
    Apply pale green highlight to selected row for visual feedback.
    
    Uses derived_virtual_selected_rows which gives row index relative to
    current page view. Returns empty list when no row selected.
    
    Args:
        derived_virtual_selected_rows: List of row indices selected on current page
            Empty list or None when no selection
            
    Returns:
        list: Style rules to apply to DataTable
            Empty list when no selection
            Single rule dict when row selected
    """
    # No highlighting when nothing selected
    if derived_virtual_selected_rows is None or len(derived_virtual_selected_rows) == 0:
        return []

    # Apply pale green background (#D4EDDA) to selected row
    return [{
        'if': {'row_index': derived_virtual_selected_rows[0]},
        'backgroundColor': '#D4EDDA',
        'color': '#155724'
    }]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    """
    Update geolocation map based on selected table row.
    
    Shows Leaflet map centered on selected animal's location with marker,
    tooltip (breed), and popup (name). When no row selected, shows message
    prompting user to select a row.
    
    Map component is remounted with unique ID on each selection change to
    ensure marker position updates correctly even after popup interaction.
    
    Args:
        viewData: List of dicts representing currently visible table data
        index: List containing index of selected row(s)
            Empty list [] when no selection
            
    Returns:
        list or html.Div: Leaflet map component or empty state message
    """
    # Handle missing data - use full dataset as fallback
    if viewData is None:
        viewData = df.to_dict('records')
    
    # Show message when no row selected (empty list or None)
    if index is None or len(index) == 0:
        return html.Div([
            html.H4('Select a row to view animal location on map',
                    style={'textAlign': 'center', 'color': '#7f8c8d', 'padding': '50px'})
        ])

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    # Create unique map ID using animal_id to force component remount
    # This ensures marker position updates even after popup has been opened
    animal_id = str(dff.iloc[row].get('animal_id', row))
    map_key = f"map-{animal_id}"
    
    # Center map on selected animal's location
    # Column 13: location_lat, Column 14: location_long
    # Column 4: breed, Column 9: name
    return [
        dl.Map(id=map_key, style={'width': '1000px', 'height': '500px'}, 
               center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tooltip (shows breed) and popup (shows name)
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Breed tooltip
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])  # Name in popup
                ])
            ])
        ])
    ]


# Run the app in tab mode for Jupyter notebook
app.run(jupyter_mode="tab")